In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./data2/BOHB/")
target_dirs = os.listdir(path)

In [3]:
def getTrial(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_idx = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
        exec_time = dict_r[3]['info']['elapsed time']
        opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']

        if cum_first is True:
            cum_exec_time_last_value = 0.0
            cum_opt_time_last_value = 0.0
            cum_first = False

        if cum_exec_time_last_value <= 86400:
            exec_times.append(exec_time)
            opt_times.append(opt_time)
            cum_exec_time.append((cum_exec_time_last_value + exec_time))
            cum_opt_time.append((cum_opt_time_last_value + opt_time))
            epochs.append(dict_r[1])
            model_idx.append(dict_r[0])
            errors.append(dict_r[3]['loss'])
            accuracies.append(dict_r[3]['info']['test accuracy'])

            cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
            cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]
            
            dict_result = dict([("cum_exec_time", cum_exec_time),
                        ("exec_time", exec_times),
                        ("opt_time", opt_times),
                        ("model_index", model_idx),
                        ("cum_opt_time", cum_opt_time),
                        ("error", errors),
                        ("accuracy", accuracies),
                        ("epoch", epochs)])
        else:
            continue
        
    return dict_result

In [4]:
bohb_results = dict()
for d in target_dirs:
    trial_num = d[5:]
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()
    
    result = [x.strip() for x in result]
    bohb_results[trial_num]=getTrial(result)

In [5]:
with open('cut_data2_result.json', 'w') as outfile:
    json.dump(bohb_results, outfile)